### Hello There!

Below is the code for the detection and characterization of benzodiazapines

The purpose of this file is to use feature extraction and traditional ML methods on our data

 Statstical features include: standard deviation, mean, median, max, min, skewness, and kurtosis

# Set up

In [ ]:
# Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import os
from IPython.display import clear_output

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
#from keras.layers.embeddings import Embedding
from keras.metrics import AUC

# Tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import random

# Import Layers
from keras.layers import ConvLSTM2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import MaxPooling3D


In [ ]:
# Hard Code Random Seeds.
r1 = 0
r2 = 1

# Set Random Seed
random.seed(r1)
tf.random.set_seed(r2)

In [ ]:
#Root folder to save data
root = "/content/drive/MyDrive/Colab Notebooks/Benzos Classification/Reference Models/TSFRESH Data/"

# Data Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Data X
Wide_X = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Benzos Classification/Data/[Smooth]SEQN_WideX.parquet')
SEQN = Wide_X['SEQN']
Wide_X = Wide_X.drop(columns=['SEQN'])

# Process X --------
data_wide = Wide_X

# Standard Scalar
scaler = StandardScaler()
scaler.fit(data_wide)
data_wide = scaler.transform(data_wide)

# Convert DF to array
data_wide = pd.DataFrame(data_wide)

In [ ]:
# Load Data Y
## Let's adjust this so not to pull in separate Y data frame; merge on SEQN
Y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Benzos Classification/Data/new_benzo_y.csv')

# Process Y --------
# Drop Unnamed Column
Y = Y.drop("Unnamed: 0", axis = 1)
Y.head()

# Change to Y Float
Y['Benzo'] = Y['y'].apply(lambda x: float(x))

#Make y array as well
y = np.hstack(np.asarray(Y.Benzo)).reshape(len(Y),1)

Numbers

In [ ]:
# Shape Analysis
print("shape of X:", data_wide.shape)
print("shape of y:", y.shape)
# Class Analysis
benzoUse = int(sum(y))
total = int(len(y))
benzoNoUse = int(total-benzoUse)

print("Benzo Use:", benzoUse, "participants")
print("Benzo No Use:", benzoNoUse, "participants")

shape of X: (7162, 10080)
shape of y: (7162, 1)
Benzo Use: 137 participants
Benzo No Use: 7025 participants


Train Test Split

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(data_wide, y, test_size=0.2, stratify=y, random_state = 19) # Hard coded random seed


# Feature Extractor

In [ ]:
def statFeatures(dataframe):
  '''
  Given a dataframe with a time series in wide format:

  Creates statistical features based off of time series
  These include: Standard Deviation, mean, median, max, min, skewness, and
  kurtosis
  '''
  # Create a feature bucket
  df = pd.DataFrame()

  # Collect features
  # df['std'] = dataframe.std(axis = 1)
  df['mean'] = dataframe.mean(axis = 1)
  df['median'] = dataframe.median(axis = 1)
  df['max'] = dataframe.max(axis = 1)
  df['min'] = dataframe.min(axis = 1)
  df['skewness'] = dataframe.skew(axis = 1)
  df['kurtosis'] = dataframe.kurt(axis = 1)
  return df

X_test = statFeatures(X_test)

# 10 fold Cross Val

In [ ]:
#@title K-Fold CV Model
%%time
from sklearn.model_selection import StratifiedKFold

# Global Score List Buckets
cv_test_scores=[]
cv_val_scores=[]

# K fold parameters
seed = 2
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# run K-fold
count = 1
for train, val in kfold.split(X_train, y_train):

  # Create New Training Set
  X_training = X_train.iloc[train]
  y_training = y_train[train]

  X_val = X_train.iloc[val]
  y_val = y_train[val]

  # Get Features
  X_training = statFeatures(X_training)
  X_val = statFeatures(X_val)

  # Fit model training
  clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_training, y_training)

  # Test the model on the Training set
  print("Training", roc_auc_score(y_training, clf.predict_proba(X_training)[:, 1]))

  # Test the model on validation set
  print("Validation", roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1]))
  cv_val_scores.append(roc_auc_score(y_val, clf.predict_proba(X_val)[:, 1]))

  # Test Model on held out test set
  print("Test", roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))
  cv_test_scores.append(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

  # increment
  print("FINISHED CYCLE NUMBER:", count)
  count += 1

# Score Eval
print("\nCV Test AUC----------------------------")
print("Individual scores:", cv_test_scores)
print("Mean:", np.mean(cv_test_scores))
print("std:", np.std(cv_test_scores))
print("\nCV Val AUC-----------------------------")
print("Individual scores:", cv_val_scores)
print("Mean:", np.mean(cv_val_scores))
print("std:", np.std(cv_val_scores))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training 0.7126735018765867
Validation 0.8154318990617924
Test 0.6858173963437122
FINISHED CYCLE NUMBER: 1


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training 0.7365088496193893
Validation 0.6112908443869298
Test 0.6884779516358464
FINISHED CYCLE NUMBER: 2


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training 0.7275124190291284
Validation 0.6894208993853121
Test 0.6883198988462146
FINISHED CYCLE NUMBER: 8


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training 0.7241747113212409
Validation 0.7246845681009383
Test 0.6902428744534008
FINISHED CYCLE NUMBER: 9
Training 0.7204408657552194
Validation 0.7476908118619349
Test 0.6890574785311627
FINISHED CYCLE NUMBER: 10

CV Test AUC----------------------------
Individual scores: [0.6858173963437122, 0.6884779516358464, 0.6885042937674516, 0.6913492439808229, 0.6877930562141088, 0.6881355039249776, 0.6899267688741373, 0.6883198988462146, 0.6902428744534008, 0.6890574785311627]
Mean: 0.6887624466571834
std: 0.0014378943685493845

CV Val AUC-----------------------------
Individual scores: [0.8154318990617924, 0.6112908443869298, 0.659657068909738, 0.6944354577806535, 0.6483338725331609, 0.7958589453251375, 0.783888709155613, 0.6894208993853121, 0.7246845681009383, 0.7476908118619349]
Mean: 0.717069307650121
std: 0.06467614834803774
CPU times: user 32.4 s, sys: 15.2 s, total: 47.6 s
Wall time: 40.2 s


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
